<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/PyTorchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This* NoteBook illustrates the examples given in chapter 1 of Inside Deep Learning Book.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd

Any value of n for an n-dimensional array is still a tensor. The word tensor is simply referring to the overall concept of an n-dimenional array.
We care about the shape of the tensor as we use different shapes to represent different types of data.

In [ ]:
import torch

torch_scalar = torch.tensor(3.14)
torch_vector = torch.tensor([1, 2, 3, 4])
torch_matrix = torch.tensor([[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12] 
                            ])
torch_tensor3d = torch.tensor([ 
                               [[1, 2, 3, 4],
                                [5, 6, 7, 8],
                                [9, 10, 11, 12] 
                               ],
                               [[13, 14, 15, 16],
                                [17, 18, 19, 20],
                                [21, 22, 23, 24] 
                               ]
                              ])

print(torch_scalar.shape)
print(torch_vector.shape)
print(torch_matrix.shape)
print(torch_tensor3d.shape)

torch.Size([])
torch.Size([4])
torch.Size([3, 4])
torch.Size([2, 3, 4])


Converion of data from numpy to Tensor, Viceversa 
Numpy defaults to 64 bit. Pytorch defaults to 32-bit floats.
But when we create 'PyTorch Tensor' from 'Numpy Array' - Tensor will use same data type as Numpy Array 



In [ ]:
x_np = np.random.random((4, 4))
print(x_np)

[[0.66627752 0.34470361 0.98000092 0.87802965]
 [0.49663691 0.0667962  0.92238922 0.89889232]
 [0.97110054 0.76772839 0.18380094 0.48625014]
 [0.37506643 0.31309093 0.91349204 0.76906552]]


In [ ]:
x_pt = torch.tensor(x_np)
print(x_pt)

tensor([[0.6663, 0.3447, 0.9800, 0.8780],
        [0.4966, 0.0668, 0.9224, 0.8989],
        [0.9711, 0.7677, 0.1838, 0.4863],
        [0.3751, 0.3131, 0.9135, 0.7691]], dtype=torch.float64)


To avoid issues with the types, you can always specify the data type to be used 'dtype' with the APIS.

In [24]:
print(x_np.dtype, x_pt.dtype)
x_np = np.asarray(x_pt, dtype=np.float32)
x_pt = torch.tensor(x_np, dtype=torch.float32)
print(x_np.dtype, x_pt.dtype)

float64 torch.float64
float32 torch.float32


Logical Operations & Similarity of the APIs between Numpy and Tensor.

In [25]:
b_np = (x_np > 0.5)
b_pt = (x_pt > 0.5)
print(b_np)
print(b_np.dtype)
print(b_pt)
print(b_pt.dtype)

[[ True False  True  True]
 [False False  True  True]
 [ True  True False False]
 [False False  True  True]]
bool
tensor([[ True, False,  True,  True],
        [False, False,  True,  True],
        [ True,  True, False, False],
        [False, False,  True,  True]])
torch.bool


Not always API are similar, one such example is transose which is explained as below. Please check always documention about API's functionality.

In [27]:
np.transpose(x_np)

array([[0.6662775 , 0.4966369 , 0.97110057, 0.37506643],
       [0.3447036 , 0.0667962 , 0.7677284 , 0.31309092],
       [0.9800009 , 0.9223892 , 0.18380095, 0.913492  ],
       [0.87802964, 0.89889234, 0.48625013, 0.7690655 ]], dtype=float32)

In [28]:
torch.transpose(x_pt, 0, 1)

tensor([[0.6663, 0.4966, 0.9711, 0.3751],
        [0.3447, 0.0668, 0.7677, 0.3131],
        [0.9800, 0.9224, 0.1838, 0.9135],
        [0.8780, 0.8989, 0.4863, 0.7691]])

First important functionality of PyTorch(Tensor) beyond what numpy can provide is to use GPU to accelarate mathematical caluclations. 
Below shows some timings related to CPU and Calculations.

In [29]:
import timeit
x = torch.rand(2**11, 2**11)
time_cpu  = timeit.timeit("x@x", globals=globals(), number=100)